# Azerbaijan Health Insurance Provider Network Analysis

This notebook analyzes healthcare provider data from 5 major insurance companies in Azerbaijan:
- **Pasha Insurance**
- **AIIC** (Azerbaijan Industrial Insurance)
- **Ateshgah Insurance**
- **Meqa Sigorta**
- **A-Group Insurance**

We examine the distribution of pharmacies, clinics, dental clinics, and optical stores across Baku and regional areas.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

# Define paths
DATA_DIR = '../data'
CHARTS_DIR = '../charts'

# Ensure charts directory exists
os.makedirs(CHARTS_DIR, exist_ok=True)

# Load combined data
df = pd.read_csv(os.path.join(DATA_DIR, 'combined.csv'))

# Display basic info
print(f"Total records: {len(df)}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nData types:\n{df.dtypes}")

In [ ]:
# Data overview
print("=" * 60)
print("DATA OVERVIEW")
print("=" * 60)
print(f"\nTotal healthcare providers: {len(df)}")
print(f"Insurance companies: {df['source'].nunique()}")
print(f"Provider types: {df['type'].nunique()}")
print(f"Cities/regions covered: {df['city'].nunique()}")
print(f"\nRecords with coordinates: {df['latitude'].notna().sum()} ({df['latitude'].notna().sum()/len(df)*100:.1f}%)")

## Chart 1: Overall Provider Distribution by Insurance Company

In [ ]:
# Chart 1: Provider count by insurance company
fig, ax = plt.subplots(figsize=(10, 6))

source_counts = df['source'].value_counts().sort_values(ascending=True)
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

bars = ax.barh(source_counts.index, source_counts.values, color=colors[:len(source_counts)])

# Add value labels
for bar, value in zip(bars, source_counts.values):
    ax.text(value + 5, bar.get_y() + bar.get_height()/2, f'{value}', 
            va='center', fontsize=11, fontweight='bold')

ax.set_xlabel('Number of Healthcare Providers')
ax.set_title('Healthcare Provider Network Size by Insurance Company', fontsize=14, fontweight='bold')
ax.set_xlim(0, max(source_counts.values) * 1.15)

plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '01_providers_by_company.png'), dpi=150, bbox_inches='tight')
plt.show()
print("Saved: 01_providers_by_company.png")

## Chart 2: Provider Types Distribution

In [ ]:
# Chart 2: Provider types distribution
fig, ax = plt.subplots(figsize=(10, 6))

type_counts = df['type'].value_counts()
colors_pie = ['#2ecc71', '#3498db', '#e74c3c', '#9b59b6']

wedges, texts, autotexts = ax.pie(type_counts.values, labels=type_counts.index, 
                                   autopct='%1.1f%%', colors=colors_pie,
                                   explode=[0.02]*len(type_counts),
                                   shadow=True, startangle=90)

# Make percentage text bold
for autotext in autotexts:
    autotext.set_fontweight('bold')
    autotext.set_fontsize(11)

ax.set_title('Distribution of Healthcare Provider Types', fontsize=14, fontweight='bold')

# Add legend with counts
legend_labels = [f'{t}: {c}' for t, c in zip(type_counts.index, type_counts.values)]
ax.legend(wedges, legend_labels, title="Provider Types", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))

plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '02_provider_types_distribution.png'), dpi=150, bbox_inches='tight')
plt.show()
print("Saved: 02_provider_types_distribution.png")

## Chart 3: Baku vs Regions Coverage

In [ ]:
# Chart 3: Baku vs Regions
fig, ax = plt.subplots(figsize=(10, 6))

region_counts = df['region_category'].value_counts()
colors_region = ['#e74c3c', '#3498db']

bars = ax.bar(region_counts.index, region_counts.values, color=colors_region, width=0.6)

# Add value labels and percentages
total = region_counts.sum()
for bar, value in zip(bars, region_counts.values):
    pct = value / total * 100
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 10, 
            f'{value}\n({pct:.1f}%)', ha='center', fontsize=12, fontweight='bold')

ax.set_ylabel('Number of Providers')
ax.set_title('Healthcare Provider Concentration: Baku vs Regions', fontsize=14, fontweight='bold')
ax.set_ylim(0, max(region_counts.values) * 1.2)

plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '03_baku_vs_regions.png'), dpi=150, bbox_inches='tight')
plt.show()
print("Saved: 03_baku_vs_regions.png")

## Chart 4: Insurance Company Coverage - Baku vs Regions

In [ ]:
# Chart 4: Company coverage by region
fig, ax = plt.subplots(figsize=(12, 6))

pivot = df.groupby(['source', 'region_category']).size().unstack(fill_value=0)
pivot = pivot.reindex(columns=['Bakı', 'Region'])

x = np.arange(len(pivot.index))
width = 0.35

bars1 = ax.bar(x - width/2, pivot['Bakı'], width, label='Bakı', color='#e74c3c')
bars2 = ax.bar(x + width/2, pivot['Region'], width, label='Region', color='#3498db')

# Add value labels
for bar in bars1:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 3, 
            f'{int(bar.get_height())}', ha='center', fontsize=10)
for bar in bars2:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 3, 
            f'{int(bar.get_height())}', ha='center', fontsize=10)

ax.set_ylabel('Number of Providers')
ax.set_title('Insurance Company Coverage: Baku vs Regions', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(pivot.index, rotation=15, ha='right')
ax.legend()

plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '04_company_baku_vs_regions.png'), dpi=150, bbox_inches='tight')
plt.show()
print("Saved: 04_company_baku_vs_regions.png")

## Chart 5: Pharmacy Network Strength by Company

In [ ]:
# Chart 5: Pharmacy network by company
fig, ax = plt.subplots(figsize=(10, 6))

pharmacy_df = df[df['type'] == 'PHARMACY']
pharmacy_counts = pharmacy_df['source'].value_counts().sort_values(ascending=True)

colors_pharmacy = plt.cm.Greens(np.linspace(0.4, 0.9, len(pharmacy_counts)))
bars = ax.barh(pharmacy_counts.index, pharmacy_counts.values, color=colors_pharmacy)

for bar, value in zip(bars, pharmacy_counts.values):
    ax.text(value + 2, bar.get_y() + bar.get_height()/2, f'{value}', 
            va='center', fontsize=11, fontweight='bold')

ax.set_xlabel('Number of Pharmacies')
ax.set_title('Pharmacy Network Strength by Insurance Company', fontsize=14, fontweight='bold')
ax.set_xlim(0, max(pharmacy_counts.values) * 1.15)

plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '05_pharmacy_network_by_company.png'), dpi=150, bbox_inches='tight')
plt.show()
print("Saved: 05_pharmacy_network_by_company.png")

## Chart 6: Clinic Network Strength by Company

In [ ]:
# Chart 6: Clinic network by company
fig, ax = plt.subplots(figsize=(10, 6))

clinic_df = df[df['type'] == 'CLINIC']
clinic_counts = clinic_df['source'].value_counts().sort_values(ascending=True)

colors_clinic = plt.cm.Blues(np.linspace(0.4, 0.9, len(clinic_counts)))
bars = ax.barh(clinic_counts.index, clinic_counts.values, color=colors_clinic)

for bar, value in zip(bars, clinic_counts.values):
    ax.text(value + 2, bar.get_y() + bar.get_height()/2, f'{value}', 
            va='center', fontsize=11, fontweight='bold')

ax.set_xlabel('Number of Clinics')
ax.set_title('Clinic Network Strength by Insurance Company', fontsize=14, fontweight='bold')
ax.set_xlim(0, max(clinic_counts.values) * 1.15)

plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '06_clinic_network_by_company.png'), dpi=150, bbox_inches='tight')
plt.show()
print("Saved: 06_clinic_network_by_company.png")

## Chart 7: Provider Type Mix by Insurance Company

In [ ]:
# Chart 7: Stacked bar - provider type mix by company
fig, ax = plt.subplots(figsize=(12, 6))

pivot_type = df.groupby(['source', 'type']).size().unstack(fill_value=0)
pivot_type_pct = pivot_type.div(pivot_type.sum(axis=1), axis=0) * 100

colors_types = {'PHARMACY': '#2ecc71', 'CLINIC': '#3498db', 'DENTAL': '#e74c3c', 'OPTICS': '#9b59b6'}

bottom = np.zeros(len(pivot_type_pct))
for col in pivot_type_pct.columns:
    ax.bar(pivot_type_pct.index, pivot_type_pct[col], bottom=bottom, 
           label=col, color=colors_types.get(col, '#7f7f7f'))
    bottom += pivot_type_pct[col].values

ax.set_ylabel('Percentage (%)')
ax.set_title('Provider Type Mix by Insurance Company (Percentage)', fontsize=14, fontweight='bold')
ax.set_xticklabels(pivot_type_pct.index, rotation=15, ha='right')
ax.legend(title='Provider Type', bbox_to_anchor=(1.02, 1), loc='upper left')
ax.set_ylim(0, 100)

plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '07_provider_type_mix_by_company.png'), dpi=150, bbox_inches='tight')
plt.show()
print("Saved: 07_provider_type_mix_by_company.png")

## Chart 8: Top 10 Cities by Provider Count

In [ ]:
# Chart 8: Top cities by provider count
fig, ax = plt.subplots(figsize=(12, 6))

city_counts = df['city'].value_counts().head(10)
colors_city = plt.cm.viridis(np.linspace(0.2, 0.8, len(city_counts)))

bars = ax.bar(range(len(city_counts)), city_counts.values, color=colors_city)

for i, (bar, value) in enumerate(zip(bars, city_counts.values)):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, 
            f'{value}', ha='center', fontsize=10, fontweight='bold')

ax.set_ylabel('Number of Providers')
ax.set_title('Top 10 Cities/Regions by Healthcare Provider Count', fontsize=14, fontweight='bold')
ax.set_xticks(range(len(city_counts)))
ax.set_xticklabels(city_counts.index, rotation=45, ha='right')

plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '08_top_cities_providers.png'), dpi=150, bbox_inches='tight')
plt.show()
print("Saved: 08_top_cities_providers.png")

## Chart 9: Regional Coverage by Provider Type

In [ ]:
# Chart 9: Regional coverage by provider type
fig, ax = plt.subplots(figsize=(10, 6))

region_df = df[df['region_category'] == 'Region']
region_type_counts = region_df['type'].value_counts()

colors_regional = ['#2ecc71', '#3498db', '#e74c3c', '#9b59b6']
wedges, texts, autotexts = ax.pie(region_type_counts.values, labels=region_type_counts.index, 
                                   autopct='%1.1f%%', colors=colors_regional[:len(region_type_counts)],
                                   explode=[0.03]*len(region_type_counts),
                                   shadow=True, startangle=90)

for autotext in autotexts:
    autotext.set_fontweight('bold')

ax.set_title('Provider Type Distribution in Regional Areas (Outside Baku)', fontsize=14, fontweight='bold')

legend_labels = [f'{t}: {c}' for t, c in zip(region_type_counts.index, region_type_counts.values)]
ax.legend(wedges, legend_labels, title="Provider Types", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))

plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '09_regional_provider_types.png'), dpi=150, bbox_inches='tight')
plt.show()
print("Saved: 09_regional_provider_types.png")

## Chart 10: Dental & Optics Specialty Coverage

In [ ]:
# Chart 10: Dental and Optics specialty coverage
fig, ax = plt.subplots(figsize=(12, 6))

specialty_df = df[df['type'].isin(['DENTAL', 'OPTICS'])]
specialty_pivot = specialty_df.groupby(['source', 'type']).size().unstack(fill_value=0)

x = np.arange(len(specialty_pivot.index))
width = 0.35

bars1 = ax.bar(x - width/2, specialty_pivot.get('DENTAL', [0]*len(x)), width, 
               label='Dental', color='#e74c3c')
bars2 = ax.bar(x + width/2, specialty_pivot.get('OPTICS', [0]*len(x)), width, 
               label='Optics', color='#9b59b6')

for bar in bars1:
    if bar.get_height() > 0:
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                f'{int(bar.get_height())}', ha='center', fontsize=10)
for bar in bars2:
    if bar.get_height() > 0:
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                f'{int(bar.get_height())}', ha='center', fontsize=10)

ax.set_ylabel('Number of Providers')
ax.set_title('Dental & Optical Care Network by Insurance Company', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(specialty_pivot.index, rotation=15, ha='right')
ax.legend()

plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '10_dental_optics_coverage.png'), dpi=150, bbox_inches='tight')
plt.show()
print("Saved: 10_dental_optics_coverage.png")

## Chart 11: Geographic Distribution Heatmap (Baku Area)

In [ ]:
# Chart 11: Geographic scatter plot for Baku
fig, ax = plt.subplots(figsize=(12, 10))

# Filter for Baku with valid coordinates
baku_df = df[(df['region_category'] == 'Bakı') & 
             (df['latitude'].notna()) & 
             (df['longitude'].notna())]

# Filter Baku bounds
baku_df = baku_df[(baku_df['latitude'] >= 40.2) & (baku_df['latitude'] <= 40.6) &
                   (baku_df['longitude'] >= 49.5) & (baku_df['longitude'] <= 50.2)]

type_colors = {'PHARMACY': '#2ecc71', 'CLINIC': '#3498db', 'DENTAL': '#e74c3c', 'OPTICS': '#9b59b6'}
type_markers = {'PHARMACY': 'o', 'CLINIC': 's', 'DENTAL': '^', 'OPTICS': 'D'}

for ptype in baku_df['type'].unique():
    subset = baku_df[baku_df['type'] == ptype]
    ax.scatter(subset['longitude'], subset['latitude'], 
               c=type_colors.get(ptype, 'gray'),
               marker=type_markers.get(ptype, 'o'),
               label=f'{ptype} ({len(subset)})', alpha=0.6, s=50)

ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Healthcare Provider Distribution in Baku Metropolitan Area', fontsize=14, fontweight='bold')
ax.legend(title='Provider Type', loc='upper right')

plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '11_baku_geographic_distribution.png'), dpi=150, bbox_inches='tight')
plt.show()
print("Saved: 11_baku_geographic_distribution.png")

## Chart 12: Regional Presence Score by Company

In [ ]:
# Chart 12: Regional presence (% of network outside Baku)
fig, ax = plt.subplots(figsize=(10, 6))

regional_pct = df.groupby('source').apply(
    lambda x: (x['region_category'] == 'Region').sum() / len(x) * 100
).sort_values(ascending=True)

colors_presence = plt.cm.RdYlGn(regional_pct.values / 100)
bars = ax.barh(regional_pct.index, regional_pct.values, color=colors_presence)

for bar, value in zip(bars, regional_pct.values):
    ax.text(value + 1, bar.get_y() + bar.get_height()/2, f'{value:.1f}%', 
            va='center', fontsize=11, fontweight='bold')

ax.set_xlabel('Percentage of Network in Regions (%)')
ax.set_title('Regional Presence: % of Provider Network Outside Baku', fontsize=14, fontweight='bold')
ax.axvline(x=20, color='red', linestyle='--', alpha=0.7, label='20% threshold')
ax.set_xlim(0, max(regional_pct.values) * 1.2)
ax.legend()

plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '12_regional_presence_score.png'), dpi=150, bbox_inches='tight')
plt.show()
print("Saved: 12_regional_presence_score.png")

## Chart 13: Provider Diversity Index

In [ ]:
# Chart 13: Provider diversity (types offered by each company)
fig, ax = plt.subplots(figsize=(10, 6))

diversity = df.groupby('source')['type'].nunique().sort_values(ascending=True)

colors_div = plt.cm.coolwarm(np.linspace(0.2, 0.8, len(diversity)))
bars = ax.barh(diversity.index, diversity.values, color=colors_div)

for bar, value in zip(bars, diversity.values):
    ax.text(value + 0.05, bar.get_y() + bar.get_height()/2, f'{value}', 
            va='center', fontsize=12, fontweight='bold')

ax.set_xlabel('Number of Provider Types Offered')
ax.set_title('Service Diversity: Provider Types per Insurance Company', fontsize=14, fontweight='bold')
ax.set_xlim(0, max(diversity.values) + 1)
ax.set_xticks(range(0, max(diversity.values) + 2))

plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '13_provider_diversity.png'), dpi=150, bbox_inches='tight')
plt.show()
print("Saved: 13_provider_diversity.png")

## Chart 14: Azerbaijan Geographic Coverage

In [ ]:
# Chart 14: Full Azerbaijan geographic spread
fig, ax = plt.subplots(figsize=(14, 10))

# Filter valid coordinates
geo_df = df[(df['latitude'].notna()) & (df['longitude'].notna())]
geo_df = geo_df[(geo_df['latitude'] >= 38) & (geo_df['latitude'] <= 42) &
                 (geo_df['longitude'] >= 44) & (geo_df['longitude'] <= 51)]

source_colors = {
    'Pasha Insurance': '#e74c3c',
    'AIIC': '#3498db', 
    'Ateshgah': '#2ecc71',
    'Meqa Sigorta': '#f39c12',
    'A-Group': '#9b59b6'
}

for source in geo_df['source'].unique():
    subset = geo_df[geo_df['source'] == source]
    ax.scatter(subset['longitude'], subset['latitude'], 
               c=source_colors.get(source, 'gray'),
               label=f'{source} ({len(subset)})', alpha=0.5, s=40)

ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Healthcare Provider Distribution Across Azerbaijan by Insurance Company', 
             fontsize=14, fontweight='bold')
ax.legend(title='Insurance Company', loc='upper left', bbox_to_anchor=(1, 1))

plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '14_azerbaijan_coverage_map.png'), dpi=150, bbox_inches='tight')
plt.show()
print("Saved: 14_azerbaijan_coverage_map.png")

## Chart 15: Market Share Summary

In [ ]:
# Chart 15: Market share summary
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

provider_types = ['PHARMACY', 'CLINIC', 'DENTAL', 'OPTICS']
colors_companies = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12', '#9b59b6']

for idx, ptype in enumerate(provider_types):
    ax = axes[idx // 2, idx % 2]
    type_df = df[df['type'] == ptype]
    
    if len(type_df) > 0:
        counts = type_df['source'].value_counts()
        wedges, texts, autotexts = ax.pie(counts.values, labels=counts.index,
                                          autopct='%1.1f%%', colors=colors_companies[:len(counts)],
                                          startangle=90)
        for autotext in autotexts:
            autotext.set_fontsize(9)
            autotext.set_fontweight('bold')
    
    ax.set_title(f'{ptype} Market Share (n={len(type_df)})', fontsize=12, fontweight='bold')

plt.suptitle('Market Share by Provider Type Across Insurance Companies', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '15_market_share_summary.png'), dpi=150, bbox_inches='tight')
plt.show()
print("Saved: 15_market_share_summary.png")

## Summary Statistics

In [ ]:
# Final summary
print("=" * 70)
print("HEALTH INSURANCE PROVIDER NETWORK - SUMMARY STATISTICS")
print("=" * 70)

print(f"\n{'OVERALL METRICS':^70}")
print("-" * 70)
print(f"Total unique healthcare providers: {len(df):,}")
print(f"Total pharmacies: {len(df[df['type']=='PHARMACY']):,}")
print(f"Total clinics: {len(df[df['type']=='CLINIC']):,}")
print(f"Total dental clinics: {len(df[df['type']=='DENTAL']):,}")
print(f"Total optical stores: {len(df[df['type']=='OPTICS']):,}")

print(f"\n{'GEOGRAPHIC DISTRIBUTION':^70}")
print("-" * 70)
baku_count = len(df[df['region_category'] == 'Bakı'])
region_count = len(df[df['region_category'] == 'Region'])
print(f"Baku metropolitan area: {baku_count:,} ({baku_count/len(df)*100:.1f}%)")
print(f"Regional areas: {region_count:,} ({region_count/len(df)*100:.1f}%)")
print(f"Cities/regions covered: {df['city'].nunique()}")

print(f"\n{'COMPANY RANKINGS':^70}")
print("-" * 70)
print("By total network size:")
for i, (source, count) in enumerate(df['source'].value_counts().items(), 1):
    print(f"  {i}. {source}: {count} providers")

print(f"\n{'KEY INSIGHTS':^70}")
print("-" * 70)
largest = df['source'].value_counts().idxmax()
print(f"1. Largest network: {largest} ({df['source'].value_counts().max()} providers)")

best_regional = df.groupby('source').apply(
    lambda x: (x['region_category'] == 'Region').sum() / len(x) * 100
).idxmax()
print(f"2. Best regional coverage: {best_regional}")

pharmacy_leader = df[df['type']=='PHARMACY']['source'].value_counts().idxmax()
print(f"3. Pharmacy network leader: {pharmacy_leader}")

clinic_leader = df[df['type']=='CLINIC']['source'].value_counts().idxmax()
print(f"4. Clinic network leader: {clinic_leader}")

print("\n" + "=" * 70)
print("All 15 charts have been saved to the 'charts' folder.")
print("=" * 70)